In [ ]:
!pip install -q transformers datasets accelerate evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 139.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 113.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files

files.upload()

Saving fine_tuned_deberta_model.zip to fine_tuned_deberta_model.zip
Saving GenericsKB_BEST_dev.csv to GenericsKB_BEST_dev.csv
Saving GenericsKB_BEST_train.csv to GenericsKB_BEST_train.csv
Saving CONGEN_dev.csv to CONGEN_dev.csv
Saving CONGEN_train.csv to CONGEN_train.csv
Buffered data was truncated after reaching the output size limit.

In [ ]:
!pip install -q evaluate --quiet

In [ ]:
import pandas as pd
from datasets import Dataset, concatenate_datasets
from transformers import Trainer, TrainingArguments, EvalPrediction
from transformers import DebertaTokenizer, DebertaForSequenceClassification
import numpy as np
import evaluate

In [ ]:
# prompt: now unzip and load this model

# Assuming the zipped model file is named 'fine_tuned_deberta_model.zip' and is already uploaded to Colab
!unzip fine_tuned_deberta_model.zip -d unzipped_model

# Load the fine-tuned model and tokenizer
# The saved model and tokenizer will be in the directory specified by the '-d' flag
unzipped_model_directory = "unzipped_model/fine_tuned_deberta_model" # Adjust if your saved model directory was different within the zip

tokenizer = DebertaTokenizer.from_pretrained(unzipped_model_directory)
model = DebertaForSequenceClassification.from_pretrained(unzipped_model_directory)

print("Model and tokenizer loaded successfully from zipped file.")

# Load data
congen_train_df, congen_dev_df = pd.read_csv('CONGEN_train.csv'), pd.read_csv('CONGEN_dev.csv')
genericskbbest_train_df, genericskbbest_dev_df = pd.read_csv('GenericsKB_BEST_train.csv'), \
                                                  pd.read_csv('GenericsKB_BEST_dev.csv')

# Combine and preprocess data
# Assuming both dataframes have 'text' and 'label' columns
# You might need to adjust column names based on your actual data
congen_train_df = congen_train_df[['Stripped Text', 'Original quantifier']]
genericskbbest_train_df = genericskbbest_train_df[['Stripped Text', 'Original quantifier']]
combined_train_df = pd.concat([congen_train_df, genericskbbest_train_df], ignore_index=True)


congen_dev_df = congen_dev_df[['Stripped Text', 'Original quantifier']]
genericskbbest_dev_df = genericskbbest_dev_df[['Stripped Text', 'Original quantifier']]
combined_dev_df = pd.concat([congen_dev_df, genericskbbest_dev_df], ignore_index=True)


# Map labels to integers
# ['all', 'every', 'most', 'many', 'some', 'few', 'no']
label_map = {'all': 0, 'every': 1, 'most': 2, 'many': 3, 'some': 4, 'few': 5, 'no': 6}
combined_train_df['Original quantifier'] = combined_train_df['Original quantifier'].map(label_map)
combined_dev_df['Original quantifier'] = combined_dev_df['Original quantifier'].map(label_map)


# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(combined_train_df)
dev_dataset = Dataset.from_pandas(combined_dev_df)

# Load tokenizer and model
# tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
# model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=7)

# Tokenize the dataset and add labels
def tokenize_function(examples):
    tokenized_text = tokenizer(examples['Stripped Text'], truncation=True, padding=True)
    # Add labels - ensure they are integers and in correct range (0-6 for 7 classes)
    tokenized_text['labels'] = examples['Original quantifier']
    return tokenized_text

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_dev_datasets = dev_dataset.map(tokenize_function, batched=True)

# Optional: Verify label distribution
print("Training label distribution:")
print(tokenized_train_datasets['labels'])
print("\nValidation label distribution:")
print(tokenized_dev_datasets['labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  # Specify eval_accuracy
    greater_is_better=True,  # Higher accuracy is better
)

# Define the metric for evaluation
metric = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_dev_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Evaluate the model and print the accuracy
train_results = trainer.evaluate(eval_dataset=tokenized_train_datasets)
eval_results = trainer.evaluate(eval_dataset=tokenized_dev_datasets)

print(f"Training Accuracy: {train_results.get('eval_accuracy')}")
print(f"Validation Accuracy: {eval_results.get('eval_accuracy')}")




# You can now use the loaded_model for inference or further evaluation
# Example:
# text_to_predict = "Birds can fly."
# inputs = loaded_tokenizer(text_to_predict, return_tensors="pt")
# outputs = loaded_model(**inputs)
# predictions = torch.argmax(outputs.logits, dim=-1)
# predicted_label_index = predictions.item()
# # Map back to original label string if needed
# inverse_label_map = {v: k for k, v in label_map.items()}
# predicted_label_string = inverse_label_map[predicted_label_index]
# print(f"Predicted label for '{text_to_predict}': {predicted_label_string}")

Archive:  fine_tuned_deberta_model.zip
   creating: unzipped_model/fine_tuned_deberta_model/
  inflating: unzipped_model/fine_tuned_deberta_model/merges.txt  
  inflating: unzipped_model/fine_tuned_deberta_model/training_args.bin  
  inflating: unzipped_model/fine_tuned_deberta_model/model.safetensors  
  inflating: unzipped_model/fine_tuned_deberta_model/vocab.json  
  inflating: unzipped_model/fine_tuned_deberta_model/config.json  
  inflating: unzipped_model/fine_tuned_deberta_model/tokenizer_config.json  
  inflating: unzipped_model/fine_tuned_deberta_model/special_tokens_map.json  
Model and tokenizer loaded successfully from zipped file.


Map:   0%|          | 0/172785 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/9094 [00:00<?, ? examples/s]

Training label distribution:
[0, 0, 4, 0, 4, 2, 2, 2, 0, 4, 4, 4, 0, 2, 0, 2, 4, 0, 2, 4, 4, 0, 0, 4, 4, 2, 4, 2, 0, 4, 0, 2, 4, 2, 2, 2, 0, 0, 2, 2, 0, 4, 0, 4, 2, 4, 0, 2, 0, 2, 0, 4, 2, 2, 4, 0, 2, 2, 2, 2, 0, 4, 2, 0, 2, 4, 0, 2, 4, 2, 0, 2, 0, 2, 0, 2, 0, 4, 2, 2, 2, 0, 2, 4, 0, 4, 0, 0, 4, 2, 2, 4, 0, 0, 0, 2, 0, 0, 4, 0, 2, 4, 0, 2, 2, 2, 2, 0, 2, 4, 0, 0, 4, 4, 4, 4, 4, 0, 4, 4, 2, 2, 4, 4, 2, 4, 4, 0, 4, 4, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 4, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 4, 4, 0, 4, 0, 0, 2, 4, 4, 2, 4, 4, 2, 4, 2, 0, 2, 4, 2, 4, 2, 4, 4, 2, 0, 0, 0, 4, 4, 2, 4, 0, 4, 2, 4, 0, 2, 0, 2, 4, 0, 2, 0, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 4, 4, 4, 2, 4, 0, 0, 2, 2, 0, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 0, 4, 0, 2, 2, 0, 2, 2, 4, 0, 0, 4, 4, 2, 0, 2, 0, 4, 0, 2, 2, 4, 2, 2, 0, 4, 2, 2, 0, 4, 2, 0, 0, 2, 4, 0, 4, 0, 0, 0, 2, 4, 4, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 4, 2, 4, 0, 0, 2, 2, 4, 4, 0, 4, 4, 2, 4, 0, 0, 4, 0, 4, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 4, 2, 0, 0, 2, 2, 0, 0

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: r0cketr1kky to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.363100,0.867279,0.692325
2,0.435500,0.905705,0.708709
3,0.386600,0.882119,0.710688


Training Accuracy: 0.9206296842897242
Validation Accuracy: 0.710688365955575


In [ ]:
# prompt: save the model and download it
import os

# Define the directory to save the model
model_save_directory = "./fine_tuned_deberta_model_v2"

# Save the model
trainer.save_model(model_save_directory)

# Zip the saved model directory
!zip -r /content/fine_tuned_deberta_model_v2.zip {model_save_directory}

# Download the zip file
# files.download('/content/fine_tuned_deberta_model_v2.zip')

  adding: fine_tuned_deberta_model_v2/ (stored 0%)
  adding: fine_tuned_deberta_model_v2/tokenizer_config.json (deflated 74%)
  adding: fine_tuned_deberta_model_v2/vocab.json (deflated 68%)
  adding: fine_tuned_deberta_model_v2/model.safetensors (deflated 7%)
  adding: fine_tuned_deberta_model_v2/merges.txt (deflated 53%)
  adding: fine_tuned_deberta_model_v2/config.json (deflated 58%)
  adding: fine_tuned_deberta_model_v2/special_tokens_map.json (deflated 83%)
  adding: fine_tuned_deberta_model_v2/training_args.bin (deflated 52%)


In [ ]:
# prompt: now unzip and load this model

# Assuming the zipped model file is named 'fine_tuned_deberta_model.zip' and is already uploaded to Colab
!unzip fine_tuned_deberta_model_v2.zip -d unzipped_model

# Load the fine-tuned model and tokenizer
# The saved model and tokenizer will be in the directory specified by the '-d' flag
unzipped_model_directory = "unzipped_model/fine_tuned_deberta_model_v2" # Adjust if your saved model directory was different within the zip

tokenizer = DebertaTokenizer.from_pretrained(unzipped_model_directory)
model = DebertaForSequenceClassification.from_pretrained(unzipped_model_directory)

print("Model and tokenizer loaded successfully from zipped file.")

# Load data
congen_train_df, congen_dev_df = pd.read_csv('CONGEN_train.csv'), pd.read_csv('CONGEN_dev.csv')
genericskbbest_train_df, genericskbbest_dev_df = pd.read_csv('GenericsKB_BEST_train.csv'), \
                                                  pd.read_csv('GenericsKB_BEST_dev.csv')

# Combine and preprocess data
# Assuming both dataframes have 'text' and 'label' columns
# You might need to adjust column names based on your actual data
congen_train_df = congen_train_df[['Stripped Text', 'Original quantifier']]
genericskbbest_train_df = genericskbbest_train_df[['Stripped Text', 'Original quantifier']]
combined_train_df = pd.concat([congen_train_df, genericskbbest_train_df], ignore_index=True)


congen_dev_df = congen_dev_df[['Stripped Text', 'Original quantifier']]
genericskbbest_dev_df = genericskbbest_dev_df[['Stripped Text', 'Original quantifier']]
combined_dev_df = pd.concat([congen_dev_df, genericskbbest_dev_df], ignore_index=True)


# Map labels to integers
# ['all', 'every', 'most', 'many', 'some', 'few', 'no']
label_map = {'all': 0, 'every': 1, 'most': 2, 'many': 3, 'some': 4, 'few': 5, 'no': 6}
combined_train_df['Original quantifier'] = combined_train_df['Original quantifier'].map(label_map)
combined_dev_df['Original quantifier'] = combined_dev_df['Original quantifier'].map(label_map)


# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(combined_train_df)
dev_dataset = Dataset.from_pandas(combined_dev_df)

# Load tokenizer and model
# tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
# model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=7)

# Tokenize the dataset and add labels
def tokenize_function(examples):
    tokenized_text = tokenizer(examples['Stripped Text'], truncation=True, padding=True)
    # Add labels - ensure they are integers and in correct range (0-6 for 7 classes)
    tokenized_text['labels'] = examples['Original quantifier']
    return tokenized_text

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_dev_datasets = dev_dataset.map(tokenize_function, batched=True)

# Optional: Verify label distribution
print("Training label distribution:")
print(tokenized_train_datasets['labels'])
print("\nValidation label distribution:")
print(tokenized_dev_datasets['labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  # Specify eval_accuracy
    greater_is_better=True,  # Higher accuracy is better
)

# Define the metric for evaluation
metric = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_dev_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Evaluate the model and print the accuracy
train_results = trainer.evaluate(eval_dataset=tokenized_train_datasets)
eval_results = trainer.evaluate(eval_dataset=tokenized_dev_datasets)

print(f"Training Accuracy: {train_results.get('eval_accuracy')}")
print(f"Validation Accuracy: {eval_results.get('eval_accuracy')}")




# You can now use the loaded_model for inference or further evaluation
# Example:
# text_to_predict = "Birds can fly."
# inputs = loaded_tokenizer(text_to_predict, return_tensors="pt")
# outputs = loaded_model(**inputs)
# predictions = torch.argmax(outputs.logits, dim=-1)
# predicted_label_index = predictions.item()
# # Map back to original label string if needed
# inverse_label_map = {v: k for k, v in label_map.items()}
# predicted_label_string = inverse_label_map[predicted_label_index]
# print(f"Predicted label for '{text_to_predict}': {predicted_label_string}")

Archive:  fine_tuned_deberta_model_v2.zip
   creating: unzipped_model/fine_tuned_deberta_model_v2/
  inflating: unzipped_model/fine_tuned_deberta_model_v2/tokenizer_config.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v2/vocab.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v2/model.safetensors  
  inflating: unzipped_model/fine_tuned_deberta_model_v2/merges.txt  
  inflating: unzipped_model/fine_tuned_deberta_model_v2/config.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v2/special_tokens_map.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v2/training_args.bin  
Model and tokenizer loaded successfully from zipped file.


Map:   0%|          | 0/172785 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/9094 [00:00<?, ? examples/s]

Training label distribution:
[0, 0, 4, 0, 4, 2, 2, 2, 0, 4, 4, 4, 0, 2, 0, 2, 4, 0, 2, 4, 4, 0, 0, 4, 4, 2, 4, 2, 0, 4, 0, 2, 4, 2, 2, 2, 0, 0, 2, 2, 0, 4, 0, 4, 2, 4, 0, 2, 0, 2, 0, 4, 2, 2, 4, 0, 2, 2, 2, 2, 0, 4, 2, 0, 2, 4, 0, 2, 4, 2, 0, 2, 0, 2, 0, 2, 0, 4, 2, 2, 2, 0, 2, 4, 0, 4, 0, 0, 4, 2, 2, 4, 0, 0, 0, 2, 0, 0, 4, 0, 2, 4, 0, 2, 2, 2, 2, 0, 2, 4, 0, 0, 4, 4, 4, 4, 4, 0, 4, 4, 2, 2, 4, 4, 2, 4, 4, 0, 4, 4, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 4, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 4, 4, 0, 4, 0, 0, 2, 4, 4, 2, 4, 4, 2, 4, 2, 0, 2, 4, 2, 4, 2, 4, 4, 2, 0, 0, 0, 4, 4, 2, 4, 0, 4, 2, 4, 0, 2, 0, 2, 4, 0, 2, 0, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 4, 4, 4, 2, 4, 0, 0, 2, 2, 0, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 0, 4, 0, 2, 2, 0, 2, 2, 4, 0, 0, 4, 4, 2, 0, 2, 0, 4, 0, 2, 2, 4, 2, 2, 0, 4, 2, 2, 0, 4, 2, 0, 0, 2, 4, 0, 4, 0, 0, 0, 2, 4, 4, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 4, 2, 4, 0, 0, 2, 2, 4, 4, 0, 4, 4, 2, 4, 0, 0, 4, 0, 4, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 4, 2, 0, 0, 2, 2, 0, 0

/tmp/ipython-input-8-3933874093.py:87: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.120500,1.423509,0.692215
2,0.263100,1.415390,0.707060
3,0.393500,1.109411,0.705300


KeyboardInterrupt: 

In [ ]:
!unzip fine_tuned_deberta_model_v2.zip -d unzipped_model

unzipped_model_directory = "unzipped_model/fine_tuned_deberta_model_v2" # Adjust if your saved model directory was different within the zip

tokenizer = DebertaTokenizer.from_pretrained(unzipped_model_directory)
model = DebertaForSequenceClassification.from_pretrained(unzipped_model_directory)

print("Model and tokenizer loaded successfully from zipped file.")

# Load data
congen_train_df, congen_dev_df = pd.read_csv('CONGEN_train.csv'), pd.read_csv('CONGEN_dev.csv')
genericskbbest_train_df, genericskbbest_dev_df = pd.read_csv('GenericsKB_BEST_train.csv'), \
                                                  pd.read_csv('GenericsKB_BEST_dev.csv')

# Combine and preprocess data
# Assuming both dataframes have 'text' and 'label' columns
# You might need to adjust column names based on your actual data
congen_train_df = congen_train_df[['Stripped Text', 'Original quantifier']]
genericskbbest_train_df = genericskbbest_train_df[['Stripped Text', 'Original quantifier']]
combined_train_df = pd.concat([congen_train_df, genericskbbest_train_df], ignore_index=True)


congen_dev_df = congen_dev_df[['Stripped Text', 'Original quantifier']]
genericskbbest_dev_df = genericskbbest_dev_df[['Stripped Text', 'Original quantifier']]
combined_dev_df = pd.concat([congen_dev_df, genericskbbest_dev_df], ignore_index=True)


# Map labels to integers
# ['all', 'every', 'most', 'many', 'some', 'few', 'no']
label_map = {'all': 0, 'every': 1, 'most': 2, 'many': 3, 'some': 4, 'few': 5, 'no': 6}
combined_train_df['Original quantifier'] = combined_train_df['Original quantifier'].map(label_map)
combined_dev_df['Original quantifier'] = combined_dev_df['Original quantifier'].map(label_map)


# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(combined_train_df)
dev_dataset = Dataset.from_pandas(combined_dev_df)

# Load tokenizer and model
# tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
# model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=7)

# Tokenize the dataset and add labels
def tokenize_function(examples):
    tokenized_text = tokenizer(examples['Stripped Text'], truncation=True, padding=True)
    # Add labels - ensure they are integers and in correct range (0-6 for 7 classes)
    tokenized_text['labels'] = examples['Original quantifier']
    return tokenized_text

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_dev_datasets = dev_dataset.map(tokenize_function, batched=True)

# Optional: Verify label distribution
print("Training label distribution:")
print(tokenized_train_datasets['labels'])
print("\nValidation label distribution:")
print(tokenized_dev_datasets['labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  # Specify eval_accuracy
    greater_is_better=True,  # Higher accuracy is better
    lr_scheduler_type="linear",  # Set scheduler type to linear
    warmup_steps=500,  # Add 500 warmup steps
)

# Define the metric for evaluation
metric = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_dev_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Evaluate the model and print the accuracy
train_results = trainer.evaluate(eval_dataset=tokenized_train_datasets)
eval_results = trainer.evaluate(eval_dataset=tokenized_dev_datasets)

print(f"Training Accuracy: {train_results.get('eval_accuracy')}")
print(f"Validation Accuracy: {eval_results.get('eval_accuracy')}")




# You can now use the loaded_model for inference or further evaluation
# Example:
# text_to_predict = "Birds can fly."
# inputs = loaded_tokenizer(text_to_predict, return_tensors="pt")
# outputs = loaded_model(**inputs)
# predictions = torch.argmax(outputs.logits, dim=-1)
# predicted_label_index = predictions.item()
# # Map back to original label string if needed
# inverse_label_map = {v: k for k, v in label_map.items()}
# predicted_label_string = inverse_label_map[predicted_label_index]
# print(f"Predicted label for '{text_to_predict}': {predicted_label_string}")

Archive:  fine_tuned_deberta_model_v2.zip
replace unzipped_model/fine_tuned_deberta_model_v2/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Model and tokenizer loaded successfully from zipped file.


Map:   0%|          | 0/172785 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/9094 [00:00<?, ? examples/s]

Training label distribution:
[0, 0, 4, 0, 4, 2, 2, 2, 0, 4, 4, 4, 0, 2, 0, 2, 4, 0, 2, 4, 4, 0, 0, 4, 4, 2, 4, 2, 0, 4, 0, 2, 4, 2, 2, 2, 0, 0, 2, 2, 0, 4, 0, 4, 2, 4, 0, 2, 0, 2, 0, 4, 2, 2, 4, 0, 2, 2, 2, 2, 0, 4, 2, 0, 2, 4, 0, 2, 4, 2, 0, 2, 0, 2, 0, 2, 0, 4, 2, 2, 2, 0, 2, 4, 0, 4, 0, 0, 4, 2, 2, 4, 0, 0, 0, 2, 0, 0, 4, 0, 2, 4, 0, 2, 2, 2, 2, 0, 2, 4, 0, 0, 4, 4, 4, 4, 4, 0, 4, 4, 2, 2, 4, 4, 2, 4, 4, 0, 4, 4, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 4, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 4, 4, 0, 4, 0, 0, 2, 4, 4, 2, 4, 4, 2, 4, 2, 0, 2, 4, 2, 4, 2, 4, 4, 2, 0, 0, 0, 4, 4, 2, 4, 0, 4, 2, 4, 0, 2, 0, 2, 4, 0, 2, 0, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 4, 4, 4, 2, 4, 0, 0, 2, 2, 0, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 0, 4, 0, 2, 2, 0, 2, 2, 4, 0, 0, 4, 4, 2, 0, 2, 0, 4, 0, 2, 2, 4, 2, 2, 0, 4, 2, 2, 0, 4, 2, 0, 0, 2, 4, 0, 4, 0, 0, 0, 2, 4, 4, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 4, 2, 4, 0, 0, 2, 2, 4, 4, 0, 4, 4, 2, 4, 0, 0, 4, 0, 4, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 4, 2, 0, 0, 2, 2, 0, 0

/tmp/ipython-input-9-292527882.py:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.132100,1.411818,0.690895
2,0.299100,1.278226,0.700242
3,0.362800,0.886906,0.712228
4,0.292700,0.967182,0.711018
5,0.257000,1.099896,0.714647


Training Accuracy: 0.9688398877217351
Validation Accuracy: 0.7146470200131955


In [ ]:
# prompt: save the model and download it
import os

# Define the directory to save the model
model_save_directory = "./fine_tuned_deberta_model_v3"

# Save the model
trainer.save_model(model_save_directory)

# Zip the saved model directory
!zip -r /content/fine_tuned_deberta_model_v3.zip {model_save_directory}

# Download the zip file
# files.download('/content/fine_tuned_deberta_model_v3.zip')

  adding: fine_tuned_deberta_model_v3/ (stored 0%)
  adding: fine_tuned_deberta_model_v3/tokenizer_config.json (deflated 74%)
  adding: fine_tuned_deberta_model_v3/vocab.json (deflated 68%)
  adding: fine_tuned_deberta_model_v3/model.safetensors (deflated 7%)
  adding: fine_tuned_deberta_model_v3/merges.txt (deflated 53%)
  adding: fine_tuned_deberta_model_v3/config.json (deflated 58%)
  adding: fine_tuned_deberta_model_v3/special_tokens_map.json (deflated 83%)
  adding: fine_tuned_deberta_model_v3/training_args.bin (deflated 52%)


In [ ]:
!unzip fine_tuned_deberta_model_v3.zip -d unzipped_model

unzipped_model_directory = "unzipped_model/fine_tuned_deberta_model_v3" # Adjust if your saved model directory was different within the zip

tokenizer = DebertaTokenizer.from_pretrained(unzipped_model_directory)
model = DebertaForSequenceClassification.from_pretrained(unzipped_model_directory)

print("Model and tokenizer loaded successfully from zipped file.")

# Load data
congen_train_df, congen_dev_df = pd.read_csv('CONGEN_train.csv'), pd.read_csv('CONGEN_dev.csv')
genericskbbest_train_df, genericskbbest_dev_df = pd.read_csv('GenericsKB_BEST_train.csv'), \
                                                  pd.read_csv('GenericsKB_BEST_dev.csv')

# Combine and preprocess data
# Assuming both dataframes have 'text' and 'label' columns
# You might need to adjust column names based on your actual data
congen_train_df = congen_train_df[['Stripped Text', 'Original quantifier']]
genericskbbest_train_df = genericskbbest_train_df[['Stripped Text', 'Original quantifier']]
combined_train_df = pd.concat([congen_train_df, genericskbbest_train_df], ignore_index=True)


congen_dev_df = congen_dev_df[['Stripped Text', 'Original quantifier']]
genericskbbest_dev_df = genericskbbest_dev_df[['Stripped Text', 'Original quantifier']]
combined_dev_df = pd.concat([congen_dev_df, genericskbbest_dev_df], ignore_index=True)


# Map labels to integers
# ['all', 'every', 'most', 'many', 'some', 'few', 'no']
label_map = {'all': 0, 'every': 1, 'most': 2, 'many': 3, 'some': 4, 'few': 5, 'no': 6}
combined_train_df['Original quantifier'] = combined_train_df['Original quantifier'].map(label_map)
combined_dev_df['Original quantifier'] = combined_dev_df['Original quantifier'].map(label_map)


# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(combined_train_df)
dev_dataset = Dataset.from_pandas(combined_dev_df)

# Load tokenizer and model
# tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
# model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=7)

# Tokenize the dataset and add labels
def tokenize_function(examples):
    tokenized_text = tokenizer(examples['Stripped Text'], truncation=True, padding=True)
    # Add labels - ensure they are integers and in correct range (0-6 for 7 classes)
    tokenized_text['labels'] = examples['Original quantifier']
    return tokenized_text

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_dev_datasets = dev_dataset.map(tokenize_function, batched=True)

# Optional: Verify label distribution
print("Training label distribution:")
print(tokenized_train_datasets['labels'])
print("\nValidation label distribution:")
print(tokenized_dev_datasets['labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    learning_rate=1e-5,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  # Specify eval_accuracy
    greater_is_better=True,  # Higher accuracy is better
    lr_scheduler_type="linear",  # Set scheduler type to linear
    warmup_steps=500,  # Add 500 warmup steps
    gradient_checkpointing=True,
)

# Define the metric for evaluation
metric = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_dev_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Evaluate the model and print the accuracy
train_results = trainer.evaluate(eval_dataset=tokenized_train_datasets)
eval_results = trainer.evaluate(eval_dataset=tokenized_dev_datasets)

print(f"Training Accuracy: {train_results.get('eval_accuracy')}")
print(f"Validation Accuracy: {eval_results.get('eval_accuracy')}")




# You can now use the loaded_model for inference or further evaluation
# Example:
# text_to_predict = "Birds can fly."
# inputs = loaded_tokenizer(text_to_predict, return_tensors="pt")
# outputs = loaded_model(**inputs)
# predictions = torch.argmax(outputs.logits, dim=-1)
# predicted_label_index = predictions.item()
# # Map back to original label string if needed
# inverse_label_map = {v: k for k, v in label_map.items()}
# predicted_label_string = inverse_label_map[predicted_label_index]
# print(f"Predicted label for '{text_to_predict}': {predicted_label_string}")

Archive:  fine_tuned_deberta_model_v3.zip
   creating: unzipped_model/fine_tuned_deberta_model_v3/
  inflating: unzipped_model/fine_tuned_deberta_model_v3/tokenizer_config.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v3/vocab.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v3/model.safetensors  
  inflating: unzipped_model/fine_tuned_deberta_model_v3/merges.txt  
  inflating: unzipped_model/fine_tuned_deberta_model_v3/config.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v3/special_tokens_map.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v3/training_args.bin  
Model and tokenizer loaded successfully from zipped file.


Map:   0%|          | 0/172785 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/9094 [00:00<?, ? examples/s]

Training label distribution:
[0, 0, 4, 0, 4, 2, 2, 2, 0, 4, 4, 4, 0, 2, 0, 2, 4, 0, 2, 4, 4, 0, 0, 4, 4, 2, 4, 2, 0, 4, 0, 2, 4, 2, 2, 2, 0, 0, 2, 2, 0, 4, 0, 4, 2, 4, 0, 2, 0, 2, 0, 4, 2, 2, 4, 0, 2, 2, 2, 2, 0, 4, 2, 0, 2, 4, 0, 2, 4, 2, 0, 2, 0, 2, 0, 2, 0, 4, 2, 2, 2, 0, 2, 4, 0, 4, 0, 0, 4, 2, 2, 4, 0, 0, 0, 2, 0, 0, 4, 0, 2, 4, 0, 2, 2, 2, 2, 0, 2, 4, 0, 0, 4, 4, 4, 4, 4, 0, 4, 4, 2, 2, 4, 4, 2, 4, 4, 0, 4, 4, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 4, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 4, 4, 0, 4, 0, 0, 2, 4, 4, 2, 4, 4, 2, 4, 2, 0, 2, 4, 2, 4, 2, 4, 4, 2, 0, 0, 0, 4, 4, 2, 4, 0, 4, 2, 4, 0, 2, 0, 2, 4, 0, 2, 0, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 4, 4, 4, 2, 4, 0, 0, 2, 2, 0, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 0, 4, 0, 2, 2, 0, 2, 2, 4, 0, 0, 4, 4, 2, 0, 2, 0, 4, 0, 2, 2, 4, 2, 2, 0, 4, 2, 2, 0, 4, 2, 0, 0, 2, 4, 0, 4, 0, 0, 0, 2, 4, 4, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 4, 2, 4, 0, 0, 2, 2, 4, 4, 0, 4, 4, 2, 4, 0, 0, 4, 0, 4, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 4, 2, 0, 0, 2, 2, 0, 0

/tmp/ipython-input-11-1120491848.py:85: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.014800,2.189339,0.705960
2,0.063800,2.375757,0.716736
3,0.072900,1.982744,0.711788
4,0.090000,1.926255,0.712998
5,0.268700,1.744135,0.714207


Training Accuracy: 0.9627629713227421
Validation Accuracy: 0.7167363096547174


In [ ]:
# prompt: save the model and download it
import os

# Define the directory to save the model
model_save_directory = "./fine_tuned_deberta_model_v4"

# Save the model
trainer.save_model(model_save_directory)

# Zip the saved model directory
!zip -r /content/fine_tuned_deberta_model_v4.zip {model_save_directory}

# Download the zip file
# files.download('/content/fine_tuned_deberta_model_v4.zip')

  adding: fine_tuned_deberta_model_v4/ (stored 0%)
  adding: fine_tuned_deberta_model_v4/tokenizer_config.json (deflated 74%)
  adding: fine_tuned_deberta_model_v4/vocab.json (deflated 68%)
  adding: fine_tuned_deberta_model_v4/model.safetensors (deflated 7%)
  adding: fine_tuned_deberta_model_v4/merges.txt (deflated 53%)
  adding: fine_tuned_deberta_model_v4/config.json (deflated 58%)
  adding: fine_tuned_deberta_model_v4/special_tokens_map.json (deflated 83%)
  adding: fine_tuned_deberta_model_v4/training_args.bin (deflated 52%)


In [ ]:
# prompt: download v2 v3 v4 models

files.download('/content/fine_tuned_deberta_model_v2.zip')
files.download('/content/fine_tuned_deberta_model_v3.zip')
files.download('/content/fine_tuned_deberta_model_v4.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!unzip fine_tuned_deberta_model_v4.zip -d unzipped_model

unzipped_model_directory = "unzipped_model/fine_tuned_deberta_model_v4" # Adjust if your saved model directory was different within the zip

tokenizer = DebertaTokenizer.from_pretrained(unzipped_model_directory)
model = DebertaForSequenceClassification.from_pretrained(unzipped_model_directory)

print("Model and tokenizer loaded successfully from zipped file.")

# Load data
congen_train_df, congen_dev_df = pd.read_csv('CONGEN_train.csv'), pd.read_csv('CONGEN_dev.csv')
genericskbbest_train_df, genericskbbest_dev_df = pd.read_csv('GenericsKB_BEST_train.csv'), \
                                                  pd.read_csv('GenericsKB_BEST_dev.csv')

# Combine and preprocess data
# Assuming both dataframes have 'text' and 'label' columns
# You might need to adjust column names based on your actual data
congen_train_df = congen_train_df[['Stripped Text', 'Original quantifier']]
genericskbbest_train_df = genericskbbest_train_df[['Stripped Text', 'Original quantifier']]
combined_train_df = pd.concat([congen_train_df, genericskbbest_train_df], ignore_index=True)


congen_dev_df = congen_dev_df[['Stripped Text', 'Original quantifier']]
genericskbbest_dev_df = genericskbbest_dev_df[['Stripped Text', 'Original quantifier']]
combined_dev_df = pd.concat([congen_dev_df, genericskbbest_dev_df], ignore_index=True)


# Map labels to integers
# ['all', 'every', 'most', 'many', 'some', 'few', 'no']
label_map = {'all': 0, 'every': 1, 'most': 2, 'many': 3, 'some': 4, 'few': 5, 'no': 6}
combined_train_df['Original quantifier'] = combined_train_df['Original quantifier'].map(label_map)
combined_dev_df['Original quantifier'] = combined_dev_df['Original quantifier'].map(label_map)


# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(combined_train_df)
dev_dataset = Dataset.from_pandas(combined_dev_df)

# Load tokenizer and model
# tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
# model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=7)

# Tokenize the dataset and add labels
def tokenize_function(examples):
    tokenized_text = tokenizer(examples['Stripped Text'], truncation=True, padding=True)
    # Add labels - ensure they are integers and in correct range (0-6 for 7 classes)
    tokenized_text['labels'] = examples['Original quantifier']
    return tokenized_text

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_dev_datasets = dev_dataset.map(tokenize_function, batched=True)

# Optional: Verify label distribution
print("Training label distribution:")
print(tokenized_train_datasets['labels'])
print("\nValidation label distribution:")
print(tokenized_dev_datasets['labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=32,
    learning_rate=1e-5,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  # Specify eval_accuracy
    greater_is_better=True,  # Higher accuracy is better
    lr_scheduler_type="linear",  # Set scheduler type to linear
    warmup_steps=500,  # Add 500 warmup steps
    gradient_checkpointing=True,
)

# Define the metric for evaluation
metric = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_dev_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Evaluate the model and print the accuracy
train_results = trainer.evaluate(eval_dataset=tokenized_train_datasets)
eval_results = trainer.evaluate(eval_dataset=tokenized_dev_datasets)

print(f"Training Accuracy: {train_results.get('eval_accuracy')}")
print(f"Validation Accuracy: {eval_results.get('eval_accuracy')}")




# You can now use the loaded_model for inference or further evaluation
# Example:
# text_to_predict = "Birds can fly."
# inputs = loaded_tokenizer(text_to_predict, return_tensors="pt")
# outputs = loaded_model(**inputs)
# predictions = torch.argmax(outputs.logits, dim=-1)
# predicted_label_index = predictions.item()
# # Map back to original label string if needed
# inverse_label_map = {v: k for k, v in label_map.items()}
# predicted_label_string = inverse_label_map[predicted_label_index]
# print(f"Predicted label for '{text_to_predict}': {predicted_label_string}")

Archive:  fine_tuned_deberta_model_v4.zip
   creating: unzipped_model/fine_tuned_deberta_model_v4/
  inflating: unzipped_model/fine_tuned_deberta_model_v4/tokenizer_config.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v4/vocab.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v4/model.safetensors  
  inflating: unzipped_model/fine_tuned_deberta_model_v4/merges.txt  
  inflating: unzipped_model/fine_tuned_deberta_model_v4/config.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v4/special_tokens_map.json  
  inflating: unzipped_model/fine_tuned_deberta_model_v4/training_args.bin  
Model and tokenizer loaded successfully from zipped file.


Map:   0%|          | 0/172785 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/9094 [00:00<?, ? examples/s]

Training label distribution:
[0, 0, 4, 0, 4, 2, 2, 2, 0, 4, 4, 4, 0, 2, 0, 2, 4, 0, 2, 4, 4, 0, 0, 4, 4, 2, 4, 2, 0, 4, 0, 2, 4, 2, 2, 2, 0, 0, 2, 2, 0, 4, 0, 4, 2, 4, 0, 2, 0, 2, 0, 4, 2, 2, 4, 0, 2, 2, 2, 2, 0, 4, 2, 0, 2, 4, 0, 2, 4, 2, 0, 2, 0, 2, 0, 2, 0, 4, 2, 2, 2, 0, 2, 4, 0, 4, 0, 0, 4, 2, 2, 4, 0, 0, 0, 2, 0, 0, 4, 0, 2, 4, 0, 2, 2, 2, 2, 0, 2, 4, 0, 0, 4, 4, 4, 4, 4, 0, 4, 4, 2, 2, 4, 4, 2, 4, 4, 0, 4, 4, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 4, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 4, 4, 0, 4, 0, 0, 2, 4, 4, 2, 4, 4, 2, 4, 2, 0, 2, 4, 2, 4, 2, 4, 4, 2, 0, 0, 0, 4, 4, 2, 4, 0, 4, 2, 4, 0, 2, 0, 2, 4, 0, 2, 0, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 4, 4, 4, 2, 4, 0, 0, 2, 2, 0, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 0, 4, 0, 2, 2, 0, 2, 2, 4, 0, 0, 4, 4, 2, 0, 2, 0, 4, 0, 2, 2, 4, 2, 2, 0, 4, 2, 2, 0, 4, 2, 0, 0, 2, 4, 0, 4, 0, 0, 0, 2, 4, 4, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 4, 2, 4, 0, 0, 2, 2, 4, 4, 0, 4, 4, 2, 4, 0, 0, 4, 0, 4, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 4, 2, 0, 0, 2, 2, 0, 0

/tmp/ipython-input-14-3032094527.py:85: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.044400,2.777457,0.711348
2,0.058800,2.521449,0.708709
3,0.081800,1.606051,0.713877
4,0.160400,1.549064,0.707280
5,0.280000,1.106168,0.711678
6,0.236900,1.212690,0.715087
7,0.088100,1.303705,0.715637
8,0.149000,1.327187,0.722015
9,0.109300,1.440991,0.725093
10,0.098300,1.536468,0.726083


Epoch,Training Loss,Validation Loss,Accuracy
1,0.044400,2.777457,0.711348
2,0.058800,2.521449,0.708709
3,0.081800,1.606051,0.713877
4,0.160400,1.549064,0.707280
5,0.280000,1.106168,0.711678
6,0.236900,1.212690,0.715087
7,0.088100,1.303705,0.715637
8,0.149000,1.327187,0.722015
9,0.109300,1.440991,0.725093
10,0.098300,1.536468,0.726083


In [ ]:
# prompt: set lr_scheduler_type="linear" and add a warm‑up of about 500 steps to smooth training

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  # Specify eval_accuracy
    greater_is_better=True,  # Higher accuracy is better

)

In [ ]:
# Load data
congen_train_df, congen_dev_df = pd.read_csv('CONGEN_train.csv'), pd.read_csv('CONGEN_dev.csv')
genericskbbest_train_df, genericskbbest_dev_df = pd.read_csv('GenericsKB_BEST_train.csv'), \
                                                  pd.read_csv('GenericsKB_BEST_dev.csv')

# Combine and preprocess data
# Assuming both dataframes have 'text' and 'label' columns
# You might need to adjust column names based on your actual data
congen_train_df = congen_train_df[['Stripped Text', 'Original quantifier']]
genericskbbest_train_df = genericskbbest_train_df[['Stripped Text', 'Original quantifier']]
combined_train_df = pd.concat([congen_train_df, genericskbbest_train_df], ignore_index=True)


congen_dev_df = congen_dev_df[['Stripped Text', 'Original quantifier']]
genericskbbest_dev_df = genericskbbest_dev_df[['Stripped Text', 'Original quantifier']]
combined_dev_df = pd.concat([congen_dev_df, genericskbbest_dev_df], ignore_index=True)


# Map labels to integers
# ['all', 'every', 'most', 'many', 'some', 'few', 'no']
label_map = {'all': 0, 'every': 1, 'most': 2, 'many': 3, 'some': 4, 'few': 5, 'no': 6}
combined_train_df['Original quantifier'] = combined_train_df['Original quantifier'].map(label_map)
combined_dev_df['Original quantifier'] = combined_dev_df['Original quantifier'].map(label_map)


# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(combined_train_df)
dev_dataset = Dataset.from_pandas(combined_dev_df)

# Load tokenizer and model
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=7)

# Tokenize the dataset and add labels
def tokenize_function(examples):
    tokenized_text = tokenizer(examples['Stripped Text'], truncation=True, padding=True)
    # Add labels - ensure they are integers and in correct range (0-6 for 7 classes)
    tokenized_text['labels'] = examples['Original quantifier']
    return tokenized_text

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_dev_datasets = dev_dataset.map(tokenize_function, batched=True)

# Optional: Verify label distribution
print("Training label distribution:")
print(tokenized_train_datasets['labels'])
print("\nValidation label distribution:")
print(tokenized_dev_datasets['labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  # Specify eval_accuracy
    greater_is_better=True,  # Higher accuracy is better
)

# Define the metric for evaluation
metric = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_dev_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Evaluate the model and print the accuracy
train_results = trainer.evaluate(eval_dataset=tokenized_train_datasets)
eval_results = trainer.evaluate(eval_dataset=tokenized_dev_datasets)

print(f"Training Accuracy: {train_results.get('eval_accuracy')}")
print(f"Validation Accuracy: {eval_results.get('eval_accuracy')}")

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/172785 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/9094 [00:00<?, ? examples/s]

Training label distribution:
[0, 0, 4, 0, 4, 2, 2, 2, 0, 4, 4, 4, 0, 2, 0, 2, 4, 0, 2, 4, 4, 0, 0, 4, 4, 2, 4, 2, 0, 4, 0, 2, 4, 2, 2, 2, 0, 0, 2, 2, 0, 4, 0, 4, 2, 4, 0, 2, 0, 2, 0, 4, 2, 2, 4, 0, 2, 2, 2, 2, 0, 4, 2, 0, 2, 4, 0, 2, 4, 2, 0, 2, 0, 2, 0, 2, 0, 4, 2, 2, 2, 0, 2, 4, 0, 4, 0, 0, 4, 2, 2, 4, 0, 0, 0, 2, 0, 0, 4, 0, 2, 4, 0, 2, 2, 2, 2, 0, 2, 4, 0, 0, 4, 4, 4, 4, 4, 0, 4, 4, 2, 2, 4, 4, 2, 4, 4, 0, 4, 4, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 4, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 4, 4, 0, 4, 0, 0, 2, 4, 4, 2, 4, 4, 2, 4, 2, 0, 2, 4, 2, 4, 2, 4, 4, 2, 0, 0, 0, 4, 4, 2, 4, 0, 4, 2, 4, 0, 2, 0, 2, 4, 0, 2, 0, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 4, 4, 4, 2, 4, 0, 0, 2, 2, 0, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 0, 4, 0, 2, 2, 0, 2, 2, 4, 0, 0, 4, 4, 2, 0, 2, 0, 4, 0, 2, 2, 4, 2, 2, 0, 4, 2, 2, 0, 4, 2, 0, 0, 2, 4, 0, 4, 0, 0, 0, 2, 4, 4, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 4, 2, 4, 0, 0, 2, 2, 4, 4, 0, 4, 4, 2, 4, 0, 0, 4, 0, 4, 0, 2, 2, 2, 4, 4, 2, 0, 2, 2, 4, 2, 0, 0, 2, 2, 0, 0

/tmp/ipython-input-22-1475798062.py:83: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.755200,0.760356,0.666044
2,0.657500,0.701743,0.704530
3,0.545900,0.723364,0.705520


Training Accuracy: 0.8185259137077872
Validation Accuracy: 0.7055201231581263


In [ ]:
# prompt: save the model and download it
import os

# Define the directory to save the model
model_save_directory = "./fine_tuned_deberta_model"

# Save the model
trainer.save_model(model_save_directory)

# Zip the saved model directory
!zip -r /content/fine_tuned_deberta_model.zip {model_save_directory}

# Download the zip file
files.download('/content/fine_tuned_deberta_model.zip')

  adding: fine_tuned_deberta_model/ (stored 0%)
  adding: fine_tuned_deberta_model/merges.txt (deflated 53%)
  adding: fine_tuned_deberta_model/training_args.bin (deflated 52%)
  adding: fine_tuned_deberta_model/model.safetensors (deflated 7%)
  adding: fine_tuned_deberta_model/vocab.json (deflated 68%)
  adding: fine_tuned_deberta_model/config.json (deflated 58%)
  adding: fine_tuned_deberta_model/tokenizer_config.json (deflated 74%)
  adding: fine_tuned_deberta_model/special_tokens_map.json (deflated 83%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

files.upload()

Saving fine_tuned_deberta_model.zip to fine_tuned_deberta_model.zip
